In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sp

In [18]:
year_range = [2010, 2022]

In [28]:
years

array([2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020,
       2021, 2022])

In [41]:
years = np.arange(year_range[0], year_range[1]+1)

for year in years:
    df2 = pd.DataFrame({})
    fname = 'gl'+str(year)
    fpath = './unsorted_game_logs/' + fname + '.txt'
    colind = np.arange(0, 11)
    colind = np.append(colind, np.array([21, 22, 49, 50]))
    colnames =  ['Date', 'Ngame', 'DOW', 'VT', 'VTL', 'VTGN', 'HT', 'HTL', 'HTGN', 'VTS', 'HTS', 
                 'VT_at_bats', 'VT_hits', 'HT_at_bats', 'HT_hits']
    gl = pd.read_csv(fpath, usecols = colind, names = colnames, index_col = False)
    teams = np.unique(gl['HT'])
    df1 = pd.DataFrame({})

    for team in teams:
        
        #Get stats for away games
        ind = np.where((gl['VT'] == team))[0]
        opponent = gl['HT'].iloc[ind]
        date = gl['Date'].iloc[ind]
        runs_scored = gl['VTS'].iloc[ind]
        runs_allowed = gl['HTS'].iloc[ind]
        game_num = gl['VTGN'].iloc[ind]
        at_bats = gl['VT_at_bats'].iloc[ind]
        hits = gl['VT_hits'].iloc[ind]
        opponent_at_bats = gl['HT_at_bats'].iloc[ind]
        opponent_hits = gl['HT_hits'].iloc[ind]
        
        #stats for home games
        ind = np.where((gl['HT'] == team))[0]
        opponent = np.append(opponent, gl['VT'].iloc[ind])
        date = np.append(date, gl['Date'].iloc[ind])
        runs_scored = np.append(runs_scored, gl['HTS'].iloc[ind])
        runs_allowed = np.append(runs_allowed, gl['VTS'].iloc[ind])
        game_num = np.append(game_num, gl['HTGN'].iloc[ind])
        at_bats = np.append(at_bats, gl['HT_at_bats'].iloc[ind])
        hits = np.append(hits, gl['HT_hits'].iloc[ind])
        opponent_at_bats = np.append(opponent_at_bats, gl['VT_at_bats'].iloc[ind])
        opponent_hits = np.append(opponent_hits, gl['VT_hits'].iloc[ind])
        
    
        df1 = pd.DataFrame({'Date': date, 'team': [team]*len(date), 'opponent': opponent, 'runs_scored': runs_scored, 
                            'runs_allowed': runs_allowed, 'game_number': game_num, 'at_bats': at_bats, 'hits': hits,
                           'opponent_at_bats': opponent_at_bats, 'opponent_hits': opponent_hits})
        
        df1 = df1.sort_values(by = ['game_number'])
        df1['win_loss'] = np.zeros(len(df1))
        
        df1 = df1.reset_index()
        
        ind = np.where(df1['runs_scored'] > df1['runs_allowed'])[0]
        df1.loc[ind, 'win_loss'] = np.ones(len(ind))
        
        win_frac_actual = np.array([])
        win_frac_cummulative_pythagorean = np.array([])
        win_frac_single_pythagorean = np.array([])
        win_frac_cummulative_pythagenpat = np.array([])
        win_frac_single_pythagenpat = np.array([])

        win_frac_act = np.zeros(len(df1))
        win_frac_cum_pyth = np.zeros(len(df1))
        win_frac_single_pyth = np.zeros(len(df1))
        exponent = np.zeros(len(df1))
        win_frac_cum_pat = np.zeros(len(df1))
        win_frac_single_pat = np.zeros(len(df1))        
        
        
        for i in range(0, len(df1)):
            win_frac_cum_pyth[i] = (sum(df1['runs_scored'].iloc[:i+1])**2)/((sum(df1['runs_scored'].iloc[:i+1])**2) + (sum(df1['runs_allowed'].iloc[:i+1])**2))
            win_frac_single_pyth[i] = (df1['runs_scored'].iloc[i]**2)/((df1['runs_scored'].iloc[i]**2) + (df1['runs_allowed'].iloc[i]**2))

            win_frac_act[i] = sum(df1['win_loss'].iloc[:i+1])/len(df1[:i+1])
            exponent = ((sum(df1['runs_scored'].iloc[:i+1]) + sum(df1['runs_allowed'].iloc[:i+1]))/sum(df1['game_number'].iloc[:i+1]))**0.287
            win_frac_cum_pat[i] = (sum(df1['runs_scored'].iloc[:i+1])**exponent)/((sum(df1['runs_scored'].iloc[:i+1])**exponent) + (sum(df1['runs_allowed'].iloc[:i+1])**exponent))
            exponent = ((df1['runs_scored'].iloc[i] + df1['runs_allowed'].iloc[i])/df1['game_number'].iloc[i])**0.287
            win_frac_single_pat[i] = (df1['runs_scored'].iloc[i]**exponent)/((df1['runs_scored'].iloc[i]**exponent) + (df1['runs_allowed'].iloc[i]**exponent))
            

        win_frac_actual = np.append(win_frac_actual, win_frac_act)
        win_frac_cummulative_pythagorean = np.append(win_frac_cummulative_pythagorean, win_frac_cum_pyth)
        win_frac_single_pythagorean = np.append(win_frac_single_pythagorean, win_frac_single_pyth)
        win_frac_cummulative_pythagenpat = np.append(win_frac_cummulative_pythagenpat, win_frac_cum_pat)
        win_frac_single_pythagenpat = np.append(win_frac_single_pythagenpat, win_frac_single_pat)       
        
        
        df1['win_frac_actual'] = win_frac_actual
        df1['win_frac_cummulative_pythagorean'] = win_frac_cummulative_pythagorean
        df1['win_frac_cummulative_pythagenpat'] = win_frac_cummulative_pythagenpat        
        
        df2 = pd.concat([df2, df1], axis = 0)
        
    save_file = './sorted_game_logs/' + fname + '_sorted.csv'
    df2.to_csv(save_file, index = False)

In [37]:
ind

array([  0,   5,   8,  10,  11,  12,  13,  14,  17,  19,  20,  22,  30,
        31,  33,  36,  37,  38,  41,  42,  44,  47,  48,  50,  51,  52,
        54,  55,  56,  57,  58,  59,  61,  63,  64,  65,  68,  69,  70,
        72,  73,  76,  77,  78,  80,  83,  88,  91,  92,  93,  95,  99,
       104, 106, 110, 111, 113, 114, 115, 117, 121, 123, 127, 131, 133,
       136, 139, 140, 141, 142, 144, 146, 148, 149, 150, 155, 156, 157,
       159, 161], dtype=int64)